# Working with Dates and Times in Python

## The Basics

In [3]:
# Modules time, datetime, calender
# Time:  
# Basic time and date functions, based on C standard
# Date/Time: As float, Precision depends on clock
# Multiple clocks -> System clock, Monotonic, Performance Counter, Process time. 
# Each clocks measures time and have different purpose/precision
# clock generator  
#   1. perf_counter 
#   2. monotonic  -> always goes forward even if system time has changed. used to measure how much time is elapsed.
# RTC (real time clock) 
#   1.process_time
#   2.clock - gets cpu clock, how much sec has the CPU worked for your program.
#   3.time - current local time, Second since 01/01/1970 
#   The function time.clock() has been removed, after having been deprecated since Python 3.3: use time.perf_counter() or time.process_time() instead, depending on your requirements

# One min is not always 60 slice
# One day is not always 24 hours
# One year is not always 365 days
# print(time.localtime())


# time
#   Basic time functionality 
#   Provides multiple clocks
#   Based on C API
# datetime
#   Basic time functionality
#   Only the “time” clock is available 
#   Object-oriented

# Use time for
#   Simple date/time operations 
#   Measuring performance 
#   Determining CPU usage 
#   Determining which event happened first - collecting stats.
#   Porting C programs
# Use datetime for
#   Most date/time operations 
#   Measuring intervals 
#   Working with local time 
#   Business logic - processing data

In [24]:
# Demo
# resolution is 10^-9 so we can trust first 9 digits after decimal point
# Assert can be raised if performace is not optimal to our expectation.

import csv
import time

class Airport:
    """ Airport as represented in airport.dat """
    prop_names = ('id','name', 'city', 'country', 'iata', 'icao','lat', 'long', 'alt', 'utc_offset', 'dst_rule', 'tz', 'type', 'source')

    def __init__(self, csv_entry) -> None:
        """ csv reader to internal dict """
        assert len(csv_entry) == len(Airport.prop_names)
        self.__dict__.update(dict(zip(Airport.prop_names, csv_entry)))
    
    def __str__(self) -> str:
        return "{0.iata} ({0.name})".format(self)

def load_airports(csv_file_name):
    """ Load airports into a dictionary where keys are IATA codes"""
    airports = {}
    with open(csv_file_name, newline='') as data_file:
        for entry in csv.reader(data_file):
            a = Airport(csv_entry=entry)
            airports[a.iata] = a
    return airports

cur_time = time.time()
print("currunt time: ", time.ctime(cur_time))
print("42 sec later time: ", time.ctime(cur_time + 42))

start_mono = time.monotonic()
start_clock = time.process_time() 

load_start = time.perf_counter()
airports = load_airports('airports.dat')
print("Loading took {0}s".format(time.perf_counter() - load_start))
print("Clock resolution: {0}s".format(time.get_clock_info('perf_counter').resolution))

def measure(f):
    start = time.perf_counter()
    f()
    return time.perf_counter() - start


print("Loading took {0}s".format(measure(lambda: load_airports('airports.dat'))))

try:
    assert(measure(lambda: load_airports('airports.dat')) < 0.01)
except AssertionError:
    print(AssertionError)

print("process_time second used ", time.process_time()-start_clock)

print("program monotonic time: ", time.monotonic() - start_mono)

print(time.localtime())

currunt time:  Wed Jul 17 15:47:55 2024
42 sec later time:  Wed Jul 17 15:48:37 2024
Loading took 0.08215964199916925s
Clock resolution: 1e-09s
Loading took 0.08708261600986589s
<class 'AssertionError'>
process_time second used  0.1764909999999995
program monotonic time:  0.23502911200921517
time.struct_time(tm_year=2024, tm_mon=7, tm_mday=17, tm_hour=15, tm_min=47, tm_sec=55, tm_wday=2, tm_yday=199, tm_isdst=0)


In [25]:
# Extention of demo to create flight schedule
import csv
import textwrap
from datetime import datetime, timedelta

class Airport:
    """ Airport as represented in airport.dat """
    prop_names = ('id','name', 'city', 'country', 'iata', 'icao','lat', 'long', 'alt', 'utc_offset', 'dst_rule', 'tz', 'type', 'source')

    def __init__(self, csv_entry) -> None:
        """ csv reader to internal dict """
        assert len(csv_entry) == len(Airport.prop_names)
        self.__dict__.update(dict(zip(Airport.prop_names, csv_entry)))
    
    def __str__(self) -> str:
        return "{0.iata} ({0.name})".format(self)

def load_airports(csv_file_name):
    """ Load airports into a dictionary where keys are IATA codes"""
    airports = {}
    with open(csv_file_name, newline='') as data_file:
        for entry in csv.reader(data_file):
            a = Airport(csv_entry=entry)
            airports[a.iata] = a
    return airports

class Flight:
    def __init__(self, flight_id, origin, destination, departure, arrival):
        self.id = flight_id
        self.origin = origin
        self.destination = destination
        self.departure = departure
        self.arrival = arrival

    @property
    def check_in(self):
        return self.departure - timedelta(hours=3)

    def time_to_departure(self):
        return self.departure - datetime.now()

    def __str__(self):
        return textwrap.dedent('''\
        Flight {0.id}:
            from        : {0.origin}
            to          : {0.destination}
            departure   : {0.departure}
            arrival     : {0.arrival}

            time to departure       : {ttd}
            check-in                : {0.check_in}
        '''.format(self, ttd = self.time_to_departure()))

airports = load_airports('airports.dat')

flights = [
    Flight(flight_id='AA123',
           origin=airports['ATL'],
           destination=airports['SVO'],
           departure=datetime(2025, 1, 1, 10, 10, 0),
           arrival=datetime(2025, 1, 2, 7, 12, 0))
]

for f in flights:
    print(f)

Flight AA123:
    from        : ATL (Hartsfield Jackson Atlanta International Airport)
    to          : SVO (Sheremetyevo International Airport)
    departure   : 2025-01-01 10:10:00
    arrival     : 2025-01-02 07:12:00

    time to departure       : 167 days, 18:17:48.505392
    check-in                : 2025-01-01 07:10:00



## Date Time Manipulation strftime and strptime

In [10]:

from datetime import datetime

now = datetime.now()
print(now.strftime('%H:%M:%S'))
print(now.strftime('%Y-%m-%d'))
print(now.strftime('%m/%d/%y'))
print(now.strftime('%j'))

print(datetime.strptime('2023', '%Y'))
try:
    datetime.strptime('hello', '%Y')
except  ValueError as e:
    print(e)


11:49:07
2024-07-17
07/17/24
199
2023-01-01 00:00:00
time data 'hello' does not match format '%Y'


## Dateutil Parser

In [17]:
from dateutil.parser import parse
dt = parse('2017')
print(dt)

dt = parse('2022 Jan 3, 18:54')
print(dt)

# Parser can take almost any string when fuzzy is true.
dt = parse('3 Janary 2017, 18:00 hello ', fuzzy=True)
print(dt)

# Default: Day/Month/Year
dt = parse('01/02/03')
print(dt)
dt = parse('01/02/03', yearfirst=True) #Year/month/day
print(dt)
dt = parse('01/02/03', yearfirst=True, dayfirst=True) #Year/day/month
print(dt)

# Lesson:
# Be Strict when formating
# Be forgiving when parsing

2017-07-17 00:00:00
2022-01-03 18:54:00
2017-03-17 18:00:00
2003-01-02 00:00:00
2001-02-03 00:00:00
2001-03-02 00:00:00


## Storing Date time in DB

### SQLite

In [2]:
# SQLite:
# Stores date/time as text
# provides some datetime functions
# python API supports datetime
# TEXT(e.g '2018-01-01 10:30:00')
# INTEGER for unix Time, rarely used as REAL
#
#                             With PARSE_DECLTYPE | Without
#              Can store datetime values directly | Stores values as strings
#    SELECT query will throw when data is invalid | Query will always succeed
#              When we are sure about data format | use otherwise
from flight_duration import Flight,airports
from recurring_flights import flights
from datetime import datetime
import sqlite3

def insert_flight(cur, flight):
    sql = 'INSERT INTO flight VALUES(?,?,?,?,?)'
    dep_naive = datetime.combine(flight.departure.date(), flight.departure.time())
    arr_naive = datetime.combine(flight.arrival.date(), flight.arrival.time())
    cur.execute(sql, (flight.id, flight.origin.iata, flight.destination.iata, dep_naive, arr_naive))

conn = sqlite3.connect(':memory:', detect_types=sqlite3.PARSE_DECLTYPES)
cur = conn.cursor()

cur.execute('CREATE TABLE flight('
            'flight_id TEXT, '
            'origin TEXT,'
            'destination TEXT,'
            'departure TIMESTAMP,'
            'arrival TIMESTAMP)')

for f in flights:
    insert_flight(cur, f)

data = cur.execute('SELECT * FROM flight').fetchall()
print(data)


for row in data[:2]:
    print(Flight(row[0], airports[row[1]], airports[row[2]], row[3], row[4]))

[('AA123', 'ATL', 'SVO', datetime.datetime(2018, 1, 1, 10, 30), datetime.datetime(2018, 1, 2, 7, 55)), ('AA123', 'ATL', 'SVO', datetime.datetime(2018, 1, 8, 10, 30), datetime.datetime(2018, 1, 9, 7, 55)), ('AA123', 'ATL', 'SVO', datetime.datetime(2018, 1, 15, 10, 30), datetime.datetime(2018, 1, 16, 7, 55)), ('AA123', 'ATL', 'SVO', datetime.datetime(2018, 1, 22, 10, 30), datetime.datetime(2018, 1, 23, 7, 55)), ('AA123', 'ATL', 'SVO', datetime.datetime(2018, 1, 29, 10, 30), datetime.datetime(2018, 1, 30, 7, 55)), ('AA123', 'ATL', 'SVO', datetime.datetime(2018, 2, 5, 10, 30), datetime.datetime(2018, 2, 6, 7, 55)), ('AA123', 'ATL', 'SVO', datetime.datetime(2018, 2, 12, 10, 30), datetime.datetime(2018, 2, 13, 7, 55)), ('AA123', 'ATL', 'SVO', datetime.datetime(2018, 2, 19, 10, 30), datetime.datetime(2018, 2, 20, 7, 55)), ('AA123', 'ATL', 'SVO', datetime.datetime(2018, 2, 26, 10, 30), datetime.datetime(2018, 2, 27, 7, 55)), ('AA123', 'ATL', 'SVO', datetime.datetime(2018, 3, 5, 10, 30), dateti

### PostgreSQL 

In [40]:
# Datetime  | PostgreSQL
# datetime  | timestamp[tz]
# time      | time[tz]
# date      | date
# timedelta | interval

import datetime
import psycopg2
from psycopg2.tz import FixedOffsetTimezone

conn = psycopg2.connect(database = "postgres", 
                        user = "postgres", 
                        host= 'localhost',
                        password = "postgres",
                        port = 5432)
curs = conn.cursor()

curs.execute("CREATE TABLE IF NOT EXISTS test_tz (t TIMESTAMP with time zone)")
conn.commit()

tz = FixedOffsetTimezone(+5*60+30, "IST")
d = datetime.datetime(1971, 10, 11, 22, 30, 0, tzinfo=tz)
# print(f"%s",(d))
curs.execute("INSERT INTO test_tz(t) VALUES (%s)", (d,))
curs.execute("SELECT * FROM test_tz LIMIT 5")
rows = curs.fetchall()
conn.commit()
conn.close()
for row in rows:
    print(row)

(datetime.datetime(1971, 10, 10, 22, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=19800))),)
(datetime.datetime(1971, 10, 10, 22, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=19800))),)
(datetime.datetime(1971, 10, 11, 22, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=19800))),)


### Mongo

In [ ]:
# Mongo
# Works directly with datetime
# Stores UTC by default
# Supports time zones
# pyMongo